# Song recommender

In [1]:
import pandas as pd
import sys
from config import *
import spotipy
import json
import pickle
from spotipy.oauth2 import SpotifyClientCredentials
pd.set_option('display.max_colwidth', None)


In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

In [3]:
cluster_df = pd.read_csv('../data/df_cluster.csv')
cluster_df

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,cluster,cluster_9,cluster_14,hot_or_not
0,0yLdNVWF3Srea0uzk55zFn,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,200455,5,4,1,H
1,3OHfY25tqY28d16oZczHc8,0.644,0.728,8,-5.750,1,0.0351,0.0543,0.169000,0.1610,0.430,88.993,153947,0,1,9,H
2,2dHHgzDwk4BJdRwy9uXhTO,0.715,0.620,1,-6.005,0,0.0484,0.4170,0.000000,0.0822,0.172,97.950,221520,3,5,12,H
3,0V3wPSX9ygBnCm8psDIegu,0.637,0.643,4,-6.571,1,0.0519,0.1300,0.000002,0.1420,0.533,97.008,200690,5,4,7,H
4,3nqQXoyQOWXiESFLlDF1hG,0.714,0.472,2,-7.375,1,0.0864,0.0130,0.000005,0.2660,0.238,131.121,156943,5,4,7,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2326,2L21CMOOqvrimKXqLIpTuX,0.842,0.681,8,-6.022,1,0.0492,0.7550,0.000062,0.2830,0.602,91.958,226440,5,1,0,N
2327,0nYiryEA1YyZWpbIxKtStQ,0.743,0.666,0,-5.466,0,0.0581,0.3080,0.000000,0.1140,0.691,149.901,205359,3,2,2,N
2328,1vFWBHNjJAPoeWcBOJk9Gp,0.658,0.742,8,-6.170,1,0.1200,0.0434,0.000000,0.2460,0.798,171.939,191263,0,1,5,N
2329,0KRAQpoakCf6ngwMTZIwfJ,0.547,0.800,5,-3.133,1,0.0345,0.0172,0.806000,0.1420,0.307,157.081,132441,6,7,11,N


In [4]:
all_songs_df = pd.read_csv('../data/all_songs_audio.csv')
all_songs_df

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artist_name
0,0yLdNVWF3Srea0uzk55zFn,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,...,117.999,audio_features,0yLdNVWF3Srea0uzk55zFn,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/audio-analysis/0yLdNVWF3Srea0uzk55zFn,200455,4,flowers,miley cyrus
1,3OHfY25tqY28d16oZczHc8,0.644,0.728,8,-5.750,1,0.0351,0.0543,0.169000,0.1610,...,88.993,audio_features,3OHfY25tqY28d16oZczHc8,spotify:track:3OHfY25tqY28d16oZczHc8,https://api.spotify.com/v1/tracks/3OHfY25tqY28d16oZczHc8,https://api.spotify.com/v1/audio-analysis/3OHfY25tqY28d16oZczHc8,153947,4,snooze,sza
2,2dHHgzDwk4BJdRwy9uXhTO,0.715,0.620,1,-6.005,0,0.0484,0.4170,0.000000,0.0822,...,97.950,audio_features,2dHHgzDwk4BJdRwy9uXhTO,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,https://api.spotify.com/v1/tracks/2dHHgzDwk4BJdRwy9uXhTO,https://api.spotify.com/v1/audio-analysis/2dHHgzDwk4BJdRwy9uXhTO,221520,4,creepin',"metro boomin, the weeknd & 21 savage"
3,0V3wPSX9ygBnCm8psDIegu,0.637,0.643,4,-6.571,1,0.0519,0.1300,0.000002,0.1420,...,97.008,audio_features,0V3wPSX9ygBnCm8psDIegu,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/audio-analysis/0V3wPSX9ygBnCm8psDIegu,200690,4,anti-hero,taylor swift
4,3nqQXoyQOWXiESFLlDF1hG,0.714,0.472,2,-7.375,1,0.0864,0.0130,0.000005,0.2660,...,131.121,audio_features,3nqQXoyQOWXiESFLlDF1hG,spotify:track:3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/tracks/3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/audio-analysis/3nqQXoyQOWXiESFLlDF1hG,156943,4,unholy,sam smith & kim petras
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2326,2L21CMOOqvrimKXqLIpTuX,0.842,0.681,8,-6.022,1,0.0492,0.7550,0.000062,0.2830,...,91.958,audio_features,2L21CMOOqvrimKXqLIpTuX,spotify:track:2L21CMOOqvrimKXqLIpTuX,https://api.spotify.com/v1/tracks/2L21CMOOqvrimKXqLIpTuX,https://api.spotify.com/v1/audio-analysis/2L21CMOOqvrimKXqLIpTuX,226440,4,thiên đàng,"wowy, jolipoli"
2327,0nYiryEA1YyZWpbIxKtStQ,0.743,0.666,0,-5.466,0,0.0581,0.3080,0.000000,0.1140,...,149.901,audio_features,0nYiryEA1YyZWpbIxKtStQ,spotify:track:0nYiryEA1YyZWpbIxKtStQ,https://api.spotify.com/v1/tracks/0nYiryEA1YyZWpbIxKtStQ,https://api.spotify.com/v1/audio-analysis/0nYiryEA1YyZWpbIxKtStQ,205359,4,vitesse,"landy, hornet la frappe"
2328,1vFWBHNjJAPoeWcBOJk9Gp,0.658,0.742,8,-6.170,1,0.1200,0.0434,0.000000,0.2460,...,171.939,audio_features,1vFWBHNjJAPoeWcBOJk9Gp,spotify:track:1vFWBHNjJAPoeWcBOJk9Gp,https://api.spotify.com/v1/tracks/1vFWBHNjJAPoeWcBOJk9Gp,https://api.spotify.com/v1/audio-analysis/1vFWBHNjJAPoeWcBOJk9Gp,191263,4,non ti dico no,"boomdabash, loredana bertè"
2329,0KRAQpoakCf6ngwMTZIwfJ,0.547,0.800,5,-3.133,1,0.0345,0.0172,0.806000,0.1420,...,157.081,audio_features,0KRAQpoakCf6ngwMTZIwfJ,spotify:track:0KRAQpoakCf6ngwMTZIwfJ,https://api.spotify.com/v1/tracks/0KRAQpoakCf6ngwMTZIwfJ,https://api.spotify.com/v1/audio-analysis/0KRAQpoakCf6ngwMTZIwfJ,132441,4,selamanya,usop


In [5]:
all_songs_df.columns

Index(['Unnamed: 0', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature', 'song_name', 'artist_name'],
      dtype='object')

In [6]:
all_songs_df['open_link'] = "https://open.spotify.com/track/"+all_songs_df['id']

In [7]:
all_songs_df.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artist_name,open_link
0,0yLdNVWF3Srea0uzk55zFn,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,...,audio_features,0yLdNVWF3Srea0uzk55zFn,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/audio-analysis/0yLdNVWF3Srea0uzk55zFn,200455,4,flowers,miley cyrus,https://open.spotify.com/track/0yLdNVWF3Srea0uzk55zFn
1,3OHfY25tqY28d16oZczHc8,0.644,0.728,8,-5.750,1,0.0351,0.0543,0.169000,0.1610,...,audio_features,3OHfY25tqY28d16oZczHc8,spotify:track:3OHfY25tqY28d16oZczHc8,https://api.spotify.com/v1/tracks/3OHfY25tqY28d16oZczHc8,https://api.spotify.com/v1/audio-analysis/3OHfY25tqY28d16oZczHc8,153947,4,snooze,sza,https://open.spotify.com/track/3OHfY25tqY28d16oZczHc8
2,2dHHgzDwk4BJdRwy9uXhTO,0.715,0.620,1,-6.005,0,0.0484,0.4170,0.000000,0.0822,...,audio_features,2dHHgzDwk4BJdRwy9uXhTO,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,https://api.spotify.com/v1/tracks/2dHHgzDwk4BJdRwy9uXhTO,https://api.spotify.com/v1/audio-analysis/2dHHgzDwk4BJdRwy9uXhTO,221520,4,creepin',"metro boomin, the weeknd & 21 savage",https://open.spotify.com/track/2dHHgzDwk4BJdRwy9uXhTO
3,0V3wPSX9ygBnCm8psDIegu,0.637,0.643,4,-6.571,1,0.0519,0.1300,0.000002,0.1420,...,audio_features,0V3wPSX9ygBnCm8psDIegu,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/audio-analysis/0V3wPSX9ygBnCm8psDIegu,200690,4,anti-hero,taylor swift,https://open.spotify.com/track/0V3wPSX9ygBnCm8psDIegu
4,3nqQXoyQOWXiESFLlDF1hG,0.714,0.472,2,-7.375,1,0.0864,0.0130,0.000005,0.2660,...,audio_features,3nqQXoyQOWXiESFLlDF1hG,spotify:track:3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/tracks/3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/audio-analysis/3nqQXoyQOWXiESFLlDF1hG,156943,4,unholy,sam smith & kim petras,https://open.spotify.com/track/3nqQXoyQOWXiESFLlDF1hG


In [8]:
all_songs_sliced = all_songs_df[['song_name','artist_name','open_link']]
all_songs_sliced.head()

,song_name,artist_name,open_link
0,flowers,miley cyrus,https://open.spotify.com/track/0yLdNVWF3Srea0uzk55zFn
1,snooze,sza,https://open.spotify.com/track/3OHfY25tqY28d16oZczHc8
2,creepin',"metro boomin, the weeknd & 21 savage",https://open.spotify.com/track/2dHHgzDwk4BJdRwy9uXhTO
3,anti-hero,taylor swift,https://open.spotify.com/track/0V3wPSX9ygBnCm8psDIegu
4,unholy,sam smith & kim petras,https://open.spotify.com/track/3nqQXoyQOWXiESFLlDF1hG


In [9]:
final_df = pd.concat([all_songs_sliced, cluster_df], axis=1)
final_df.head()

,song_name,artist_name,open_link,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,cluster,cluster_9,cluster_14,hot_or_not
0,flowers,miley cyrus,https://open.spotify.com/track/0yLdNVWF3Srea0uzk55zFn,0yLdNVWF3Srea0uzk55zFn,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,200455,5,4,1,H
1,snooze,sza,https://open.spotify.com/track/3OHfY25tqY28d16oZczHc8,3OHfY25tqY28d16oZczHc8,0.644,0.728,8,-5.750,1,0.0351,0.0543,0.169000,0.1610,0.430,88.993,153947,0,1,9,H
2,creepin',"metro boomin, the weeknd & 21 savage",https://open.spotify.com/track/2dHHgzDwk4BJdRwy9uXhTO,2dHHgzDwk4BJdRwy9uXhTO,0.715,0.620,1,-6.005,0,0.0484,0.4170,0.000000,0.0822,0.172,97.950,221520,3,5,12,H
3,anti-hero,taylor swift,https://open.spotify.com/track/0V3wPSX9ygBnCm8psDIegu,0V3wPSX9ygBnCm8psDIegu,0.637,0.643,4,-6.571,1,0.0519,0.1300,0.000002,0.1420,0.533,97.008,200690,5,4,7,H
4,unholy,sam smith & kim petras,https://open.spotify.com/track/3nqQXoyQOWXiESFLlDF1hG,3nqQXoyQOWXiESFLlDF1hG,0.714,0.472,2,-7.375,1,0.0864,0.0130,0.000005,0.2660,0.238,131.121,156943,5,4,7,H


In [10]:
final_df.rename(columns={'Unnamed: 0':'id'}, inplace=True) 
final_df.head()

,song_name,artist_name,open_link,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,cluster,cluster_9,cluster_14,hot_or_not
0,flowers,miley cyrus,https://open.spotify.com/track/0yLdNVWF3Srea0uzk55zFn,0yLdNVWF3Srea0uzk55zFn,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,200455,5,4,1,H
1,snooze,sza,https://open.spotify.com/track/3OHfY25tqY28d16oZczHc8,3OHfY25tqY28d16oZczHc8,0.644,0.728,8,-5.750,1,0.0351,0.0543,0.169000,0.1610,0.430,88.993,153947,0,1,9,H
2,creepin',"metro boomin, the weeknd & 21 savage",https://open.spotify.com/track/2dHHgzDwk4BJdRwy9uXhTO,2dHHgzDwk4BJdRwy9uXhTO,0.715,0.620,1,-6.005,0,0.0484,0.4170,0.000000,0.0822,0.172,97.950,221520,3,5,12,H
3,anti-hero,taylor swift,https://open.spotify.com/track/0V3wPSX9ygBnCm8psDIegu,0V3wPSX9ygBnCm8psDIegu,0.637,0.643,4,-6.571,1,0.0519,0.1300,0.000002,0.1420,0.533,97.008,200690,5,4,7,H
4,unholy,sam smith & kim petras,https://open.spotify.com/track/3nqQXoyQOWXiESFLlDF1hG,3nqQXoyQOWXiESFLlDF1hG,0.714,0.472,2,-7.375,1,0.0864,0.0130,0.000005,0.2660,0.238,131.121,156943,5,4,7,H


In [11]:
final_df.to_csv('../data/final_df.csv',index=False)

In [12]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as file: 
            return pickle.load(file) 
    except FileNotFoundError: 
        print("File not found!")

In [13]:
scaler = load("../Scaler/scaler.pickle")
scaler

StandardScaler()

In [14]:
model = load("../K-Means/kmeans_14.pickle")
model

KMeans(n_clusters=14, random_state=0, verbose=1)

In [15]:
def get_song_id(song_name, artist_name):
    try: 
        results=sp.search(q='tracks:' +song_name+"artist"+artist_name,limit = 1)
        return results['tracks']['items'][0]['id']
    except: 
        return "None"

In [16]:
def make_clickable(val):
    return f'<a target="_blank" href="{val}">{val}</a>'


In [17]:
def song_recommender():
    response = "Yes"
    all_my_dict={}
    while response == "Yes":
        
        title = str(input("Enter the title of your song: "))
        artist = str(input("From which artist do you want to make my search? "))
        
        user_id = get_song_id(title, artist)

        user_features = sp.audio_features(user_id)[0]
        all_my_dict[user_id] = user_features
        user_features_df = pd.DataFrame(all_my_dict).T
        user_features_df = user_features_df[['danceability', 'energy', 'key', 'loudness', 
                                             'mode','speechiness', 'acousticness', 'instrumentalness', 'liveness',
                                             'valence', 'tempo','duration_ms']]

        user_features_scaled_np = scaler.transform(user_features_df)
        user_features_scaled_df = pd.DataFrame(user_features_scaled_np, columns = user_features_df.columns)
        user_song_cluster = model.predict(user_features_scaled_df)[0]
        number = int(input('How many recommendations do you want? '))

        try:
            if user_id == final_df[final_df['id'] == user_id].loc[:,'id'].values[0]:
                reco = final_df[(final_df['hot_or_not']=="H")]
                recommendation = reco[reco['cluster_14']==user_song_cluster].sample(number)

        except:
            reco = final_df[(final_df['hot_or_not']=="N")]
            recommendation = reco[reco['cluster_14']==user_song_cluster].sample(number)

        recommendation = recommendation[['song_name','artist_name','open_link']].reset_index()
        recommendation = recommendation.drop(['index'], axis=1)
        recommendation = recommendation.style.format({'open_link': make_clickable})
        display(recommendation)

        print()
        response = str(input('Do you want any other recommendation? (Please input Yes or No): '))
        if response == "Yes":
            same = str(input('Do you want to try with another song or the same? (Please input Same or Another): '))
            if same == "Another":
                continue

            elif same == 'Same':
                more = "No"
                while more == "No":
                    number = int(input('How many recommendations do you want? '))
                    recommendation = reco[reco['cluster_14']==user_song_cluster].sample(number)
                    recommendation = recommendation[['song_name','artist_name','open_link']].reset_index()
                    recommendation = recommendation.drop(['index'], axis=1)
                    recommendation = recommendation.style.format({'open_link': make_clickable})
                    display(recommendation)
                    more = str(input("Do you want to try with another song? (Please input Yes or No): "))
        else:
            print('Thanks for using our song recomender!')

In [18]:
song_recommender()

Enter the title of your song: hello
From which artist do you want to make my search? adele
How many recommendations do you want? 7


,song_name,artist_name,open_link
0,aloha,"maluma, beéle, rauw alejandro, mambo kingz, dj luian, darell",https://open.spotify.com/track/1nVAXNncJum2YgmVRINs6v
1,scorpion,fredo,https://open.spotify.com/track/5Ovgj24PMwwCPrcSSeSV9i
2,better,"lil yachty, stefflon don",https://open.spotify.com/track/2SbFVYtBEYQ0JCHf2eiPBT
3,the 1,taylor swift,https://open.spotify.com/track/6mLTvsDMs1gTsZmUsxbRdh
4,jungle,mellemfingamuzik,https://open.spotify.com/track/0VqYs76GajViLPEfuVUrUg
5,mueve la cadera,"el alfa, chael produciendo, kiko el crazy",https://open.spotify.com/track/2fiNbfoUP150yrZom2xdgA
6,la zone,branco,https://open.spotify.com/track/459KrgRr9DQsStW75fQnVv



Do you want any other recommendation? (Please input Yes or No): Yes
Do you want to try with another song or the same? (Please input Same or Another): Same
How many recommendations do you want? 3


,song_name,artist_name,open_link
0,aloha,"maluma, beéle, rauw alejandro, mambo kingz, dj luian, darell",https://open.spotify.com/track/1nVAXNncJum2YgmVRINs6v
1,200 mph,"bad bunny, diplo, nitti gritti",https://open.spotify.com/track/0BcI1eOlSNoXEQCPlkFIG3
2,hot coffee,schafter,https://open.spotify.com/track/2CpsEzj7bkPfjC4tmU3oAM


Do you want to try with another song? (Please input Yes or No): Yes
Enter the title of your song: flowers
From which artist do you want to make my search? miley cyrus
How many recommendations do you want? 4


,song_name,artist_name,open_link
0,astronomia 2k19 - radio edit,stephan f,https://open.spotify.com/track/3hWfKBt3n7j1xqIy6LA5ve
1,bibia be ye ye,ed sheeran,https://open.spotify.com/track/6XCt3N0rnt3vdT9fSY4e9n
2,200 mph,"bad bunny, diplo, nitti gritti",https://open.spotify.com/track/0BcI1eOlSNoXEQCPlkFIG3
3,baila conmigo (feat. kelly ruiz),"dayvi, victor cardenas, kelly ruíz",https://open.spotify.com/track/4Clmg1g8n2yBYG13Z1oSdG



Do you want any other recommendation? (Please input Yes or No): No
Thanks for using our song recomender!
